In [ ]:
!pip install feature_engine
!pip install optuna
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.0/230.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 7.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from feature_engine.encoding import CountFrequencyEncoder
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,TargetEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score,cross_validate
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import r2_score, root_mean_squared_error
from scipy.stats import randint
from time import time
import optuna
from catboost import Pool, CatBoostRegressor

# Business Understanding

##### This dataset contains detailed transactional records of global chocolate product sales from July 2022 to December 2025. It is designed for sales performance analysis, trend forecasting, and individual/regional contribution evaluation in Power BI or other analytics tools.

# Analytical Approach

##### Predictive Analysis

# Data Requirement

##### This dataset contains detailed transactional records of global chocolate product sales from July 2022 to December 2025. It is designed for sales performance analysis, trend forecasting, and individual/regional contribution evaluation in Power BI or other analytics tools.

# Data Collection

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/DS_Projects/Practice_data_sets/Chocolate Sales.csv")

In [ ]:
data.head()

,Sales Person,Country,Product,Date,Amount,Boxes Shipped
0,Jehu Rudeforth,UK,Mint Chip Choco,04/01/2022,"$5,320.00",180
1,Van Tuxwell,India,85% Dark Bars,01/08/2022,"$7,896.00",94
2,Gigi Bohling,India,Peanut Butter Cubes,07/07/2022,"$4,501.00",91
3,Jan Morforth,Australia,Peanut Butter Cubes,27/04/2022,"$12,726.00",342
4,Jehu Rudeforth,UK,Peanut Butter Cubes,24/02/2022,"$13,685.00",184


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Understanding

In [ ]:
data.shape

(3282, 6)

In [ ]:
data.size

19692

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3282 entries, 0 to 3281
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Sales Person   3282 non-null   object
 1   Country        3282 non-null   object
 2   Product        3282 non-null   object
 3   Date           3282 non-null   object
 4   Amount         3282 non-null   object
 5   Boxes Shipped  3282 non-null   int64 
dtypes: int64(1), object(5)
memory usage: 154.0+ KB


In [ ]:
data.describe(include="all")

,Sales Person,Country,Product,Date,Amount,Boxes Shipped
count,3282,3282,3282,3282,3282,3282.000000
unique,25,6,22,504,3013,NaN
top,Kelci Walkden,Australia,Eclairs,13/01/2024,"$2,317.00",NaN
freq,162,615,180,16,5,NaN
mean,NaN,NaN,NaN,NaN,NaN,164.666971
std,NaN,NaN,NaN,NaN,NaN,124.024736
min,NaN,NaN,NaN,NaN,NaN,1.000000
25%,NaN,NaN,NaN,NaN,NaN,71.000000
50%,NaN,NaN,NaN,NaN,NaN,137.000000
75%,NaN,NaN,NaN,NaN,NaN,232.000000


In [ ]:
data.isna().sum()

,0
Sales Person,0
Country,0
Product,0
Date,0
Amount,0
Boxes Shipped,0


# Data Preparation

In [ ]:
data["Amount"] = data["Amount"].str.replace(r'[^\d.]', '', regex=True).astype(float)

In [ ]:
data["Date"] = pd.to_datetime(data["Date"],format='mixed')

In [ ]:
data["Day_of_week"] = data["Date"].dt.day_of_week

In [ ]:
data["Month"] = data["Date"].dt.month

In [ ]:
data["Year"] = data["Date"].dt.year

In [ ]:
data = data.drop("Date",axis=1)

In [ ]:
product_category_map = {
    '85% Dark Bars': 'Dark Chocolate',
    '99% Dark & Pure': 'Dark Chocolate',
    '70% Dark Bites': 'Dark Chocolate',
    '50% Dark Bites': 'Dark Chocolate',


    'Milk Bars': 'Milk Chocolate',
    'White Choc': 'Milk Chocolate',
    'Eclairs': 'Milk Chocolate',

    'Orange Choco': 'Flavoured Chocolate',
    'Mint Chip Choco': 'Flavoured Chocolate',
    'Raspberry Choco': 'Flavoured Chocolate',
    'Smooth Sliky Salty': 'Flavoured Chocolate',
    'Spicy Special Slims': 'Flavoured Chocolate',

    'Almond Choco': 'Nut-based',
    'Fruit & Nut Bars': 'Nut-based',
    'Choco Coated Almonds': 'Nut-based',

    'Caramel Stuffed Bars': 'Filled / Stuffed',
    'Peanut Butter Cubes': 'Filled / Stuffed',
    'Manuka Honey Choco': 'Filled / Stuffed',

    'Drinking Coco': 'Baking / Drink',
    "Baker's Choco Chips": 'Baking / Drink',
    'Organic Choco Syrup': 'Baking / Drink',

    "After Nines": 'Dark Chocolate / filled',
}


In [ ]:
def agg_func(x):
    return product_category_map[x]

In [ ]:
data["Product"] = data["Product"].map(agg_func)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3282 entries, 0 to 3281
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Sales Person   3282 non-null   object 
 1   Country        3282 non-null   object 
 2   Product        3282 non-null   object 
 3   Amount         3282 non-null   float64
 4   Boxes Shipped  3282 non-null   int64  
 5   Day_of_week    3282 non-null   int32  
 6   Month          3282 non-null   int32  
 7   Year           3282 non-null   int32  
dtypes: float64(1), int32(3), int64(1), object(3)
memory usage: 166.8+ KB


### Feature Encoding

##### one hot encoding

In [ ]:
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

In [ ]:
X_encoded = encoder.fit_transform(data[["Product"]])

In [ ]:
feature_names = encoder.get_feature_names_out(["Product"])

In [ ]:
ohe_df = pd.DataFrame(X_encoded, columns=feature_names, index=data.index)

In [ ]:
data = pd.concat([ohe_df,data],axis=1)

In [ ]:
data.drop("Product",axis=1,inplace= True)

##### Target encoding

In [ ]:
data[['Sales Person','Country']] = TargetEncoder(smooth=10).fit_transform(data[['Sales Person','Country']], data['Amount'])

##### Label encoding

In [ ]:
# le = LabelEncoder()

In [ ]:
# label_encoded_sp = le.fit_transform(data['Sales Person'])

In [ ]:
# data['Sales Person'] = pd.DataFrame(label_encoded_sp)

In [ ]:
# label_encoded_country = le.fit_transform(data['Product'])

In [ ]:
# data['Product'] = pd.DataFrame(label_encoded_country)

##### Frequency encoding

In [ ]:
# encoder = CountFrequencyEncoder(
# encoding_method='frequency',
# variables=[],
# ignore_format=True,
# )

In [ ]:
# data = encoder.fit_transform(data)

In [ ]:
data

# EDA

In [ ]:
sns.pairplot(data)
plt.show()

In [ ]:
corr = data.corr()

In [ ]:
sns.heatmap(corr, annot=True)

In [ ]:
X = data.drop(['Amount'],axis=1)
y = data['Amount']

### Train,Test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42)

In [ ]:
train_pool = Pool(X_train,
                  y_train,
                  cat_features=[0,1,2])
test_pool = Pool(X_test,
                 cat_features=[0,1,2])

### Cat Boost

Best hyperparameters found:
  learning_rate: 0.2989671588287381
  depth: 10
  l2_leaf_reg: 4.224023729483935
  min_data_in_leaf: 41
  bagging_temperature: 0.5603647297740325
  random_strength: 1.2005090691190556
  rsm: 0.9490090125560535
  grow_policy: SymmetricTree
  border_count: 39

In [ ]:
params = {
        # Core learning
        "iterations": 1200,
        "learning_rate": 0.2989671588287381,
        "depth": 10,

        # Regularization (VERY IMPORTANT)
        "l2_leaf_reg": 4.224023729483935,
        "min_data_in_leaf": 41,

        # Randomness & bagging
        "bagging_temperature": 0.5603647297740325,
        "random_strength": 1.2005090691190556,

        # Feature sampling
        "rsm": 0.9490090125560535,

        # Tree structure
        "grow_policy": "SymmetricTree",

        # Feature binning
        "border_count":39,

        # Fixed
        "loss_function": "RMSE",
        "eval_metric": "RMSE",
        "early_stopping_rounds": 50,
        "verbose": False
    }


model = CatBoostRegressor(**params)

model.fit(train_pool)
# make the prediction using the resulting model
model.score(X_test, y_test)

np.float64(0.5804265890281808)

In [ ]:
y_preds = model.predict(test_pool)

In [ ]:
r2_value=r2_score(y_test,y_preds)
rmse=root_mean_squared_error(y_test,y_preds)
print(r2_value,rmse)

0.5804265890281808 2657.187879209106


### Linear Model

In [ ]:
lm = LinearRegression()

In [ ]:
lm.fit( X_train, y_train)

In [ ]:
lm.score(X_test, y_test)

In [ ]:
scoring=('r2', 'neg_root_mean_squared_error')

In [ ]:
cv_results = cross_validate(lm, X_train, y_train, cv=5, scoring = scoring)

In [ ]:
cv_results

### TF DF Model

In [ ]:
rf= RandomForestRegressor(bootstrap= False, max_depth= 50, max_features ='sqrt', min_samples_leaf=1 , min_samples_split= 10, n_estimators= 994, random_state=42)

In [ ]:
rf.fit(X_train, y_train)

In [ ]:
rf.score( X_train, y_train)

In [ ]:
rf.score( X_test, y_test)

In [ ]:
y_pred=rf.predict( X_test)

In [ ]:
r2_value=r2_score(y_test,y_pred)
rmse=root_mean_squared_error(y_pred,y_test)

In [ ]:
print(r2_value,rmse)

In [ ]:
cv_results_rf = cross_validate(rf, X_train, y_train, cv=5, scoring = scoring)

In [ ]:
cv_results_rf

# GB Regressor

In [ ]:
gb = GradientBoostingRegressor(random_state=0)

In [ ]:
gb.fit( X_train, y_train)

In [ ]:
gb.score(X_train, y_train)

In [ ]:
gb.score(X_test, y_test)

In [ ]:
cv_results_gb = cross_validate(gb, X_train, y_train, cv=5, scoring = scoring)

In [ ]:
cv_results_gb

#### Find Best parameters for the Random Forest Regressor

In [ ]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results["rank_test_score"] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print(
                "Mean validation score: {0:.3f} (std: {1:.3f})".format(
                    results["mean_test_score"][candidate],
                    results["std_test_score"][candidate],
                )
            )
            print("Parameters: {0}".format(results["params"][candidate]))
            print("")

In [ ]:
param_dist = {
    # Number of trees in the forest (more usually better, but slower)
    'n_estimators': randint(10, 1000),

    # Number of trees in the forest (more usually better, but slower)
    'random_state': randint(0, 200),

    # Maximum depth of each tree (None means unlimited growth)
    'max_depth': [None] + list(np.arange(5, 51, 5)),

    # Number of features to consider for the best split
    'max_features': ['sqrt', 'log2', None],

    # Minimum samples required to split an internal node
    'min_samples_split': randint(2, 21),

    # Minimum samples required at each leaf node
    'min_samples_leaf': randint(1, 21),

    # Whether bootstrap samples are used when building trees
    'bootstrap': [True, False]
}

# run randomized search
n_iter_search = 15
random_search = RandomizedSearchCV(
    rf, param_distributions=param_dist, n_iter=n_iter_search
)

start = time()
random_search.fit(X_train, y_train)

In [ ]:
report(random_search.cv_results_)

In [ ]:
def objective(trial):

    params = {
        "iterations": trial.suggest_int("iterations", 300, 1500, step=100),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "depth": trial.suggest_int("depth", 4, 10),

        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1.0, 20.0, log=True),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 50),

        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 1.0),
        "random_strength": trial.suggest_float("random_strength", 0.0, 2.0),
        "rsm": trial.suggest_float("rsm", 0.6, 1.0),

        "grow_policy": trial.suggest_categorical(
            "grow_policy",
            ["SymmetricTree", "Depthwise", "Lossguide"]
        ),

        "border_count": trial.suggest_int("border_count", 32, 255),

        "loss_function": "RMSE",
        "eval_metric": "RMSE",
        "early_stopping_rounds": 50,
        "verbose": False
    }

    model = CatBoostRegressor(**params)

    model.fit(train_pool)
    score = model.score(X_test, y_test)
    # rmse = model.get_best_score()["learn"]["RMSE"]
    return score

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=55)
print(f"Number of finished trials: {len(study.trials)}")
print(f"Best trial value (accuracy): {study.best_value:.4f}")
print("Best hyperparameters found:")
for key, value in study.best_params.items():
    print(f"  {key}: {value}")

[I 2026-01-08 09:22:51,458] A new study created in memory with name: no-name-db6d141e-0d36-47f4-9979-0a58f020befa
[I 2026-01-08 09:23:01,581] Trial 0 finished with value: 0.6650023254238773 and parameters: {'iterations': 700, 'learning_rate': 0.055554387199631426, 'depth': 10, 'l2_leaf_reg': 1.4355569164284632, 'min_data_in_leaf': 28, 'bagging_temperature': 0.8840728861263758, 'random_strength': 0.04667895518670062, 'rsm': 0.7453011240353196, 'grow_policy': 'SymmetricTree', 'border_count': 89}. Best is trial 0 with value: 0.6650023254238773.
[I 2026-01-08 09:23:07,436] Trial 1 finished with value: 0.5633607600475509 and parameters: {'iterations': 400, 'learning_rate': 0.03980675111758899, 'depth': 10, 'l2_leaf_reg': 2.9865838053492255, 'min_data_in_leaf': 13, 'bagging_temperature': 0.5669247446311055, 'random_strength': 0.5680059423480015, 'rsm': 0.8541648758151185, 'grow_policy': 'SymmetricTree', 'border_count': 125}. Best is trial 0 with value: 0.6650023254238773.
[I 2026-01-08 09:23

Number of finished trials: 55
Best trial value (accuracy): 0.6894
Best hyperparameters found:
  iterations: 1200
  learning_rate: 0.12954074003772073
  depth: 7
  l2_leaf_reg: 1.3706986060989241
  min_data_in_leaf: 33
  bagging_temperature: 0.04238176787232101
  random_strength: 0.3839724179143472
  rsm: 0.6478742051045705
  grow_policy: SymmetricTree
  border_count: 208
